Привет Марк, меня зовут Артем. Сегодня я проверю твой проект.

Комментарии будут в <font color='green'>зеленой</font>, <font color='blue'>синей</font> или <font color='red'>красной</font> рамках:

<div class="alert alert-block alert-success">
<b>Успех:</b> Если все сделано отлично
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> Если можно немного улучшить
</div>

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Если требуются исправления. Работа не может быть принята с красными комментариями.
</div>

### <font color='orange'>Общее впечатление</font>
* Большое спасибо за проделанную работу. Проект сделан хорошо, видно, что приложено много усилий.
* Молодец, что при помощи markdown ячеек структурируешь ноутбук. Приятно проверять такие проекты.
* Критических замечаний нет, однако есть несколько советов. Надеюсь они помогут тебе сделать твои следующие проекты лучше.
* Очень рад, что было сделано бонусное задание. Мало у кого оно получается верным с первого раза.
* Проект зачтен, удачи в дальнейшем обучении!

# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

#### Описание данных
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## 1. Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что все импорты собраны в первой ячейке ноутбука! Если у того, кто будет запускать твой ноутбук будут отсутствовать некоторые библиотеки, то он это увидит сразу, а не в процессе!
</div>

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [4]:
df.info()
display(df.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


<div class="alert alert-block alert-success">
<b>Успех:</b> Первый шаг сделан хорошо!
</div>

## 2. Разбейте данные на выборки

Разделим данные на тренировочную, валидационную и тестовую выборки в пропорции 3:1:1. 

Разделим их следующим образом: 
1. Первым шагом разделим выборки на две: тренировочную + валидационную и тестовую. (В будущем используем тренировочную + валидационную выборку для обучения итоговой модели)
2. Вторым шагом разделим тренировочную + валидационную выборку на две соответствующие части. 

В получившихся выборках выделим обучающие признаки и целевой признак.

In [89]:
rnd_seed= 12345678

train_total_df, test_df = train_test_split(df, test_size = 0.2, random_state = rnd_seed)
train_df, valid_df = train_test_split(train_total_df, test_size = 0.25, random_state = rnd_seed)

train_total_features = train_total_df.drop('is_ultra', axis = 1)
train_total_target = train_total_df['is_ultra']

train_features = train_df.drop('is_ultra', axis = 1)
train_target = train_df['is_ultra']

valid_features = valid_df.drop('is_ultra', axis = 1)
valid_target = valid_df['is_ultra']

test_features = test_df.drop('is_ultra', axis = 1)
test_target = test_df['is_ultra']

#### Вывод
Данные разбиты на выборки. В выборках выделены признаки для обучения.

<div class="alert alert-block alert-info">
<b>Совет: </b> Все сделано корректно. Советую в следующий раз выводить размеры получившихся таблиц. Это поможет избежать ошибок.
</div>

## 3. Исследуйте модели

Исследуем модели, основанные на принципе древа решений, случайного леса и логисти. В первых двух будем изменять гиперпараметры глубины древа и количества деревьев соответственно в циклах.

In [112]:
#desigion_tree
print('Desigion Tree:')
for n in range (1, 7):
    model1 = DecisionTreeClassifier(random_state = rnd_seed, max_depth = n)
    model1.fit(train_features, train_target)
    valid_predictions1 = model1.predict(valid_features)
    print('Accuracy score for n = {}:'.format(n), accuracy_score(valid_target, valid_predictions1))

print('\n---------------------------------------------\n')

#random_forest
print('Random Forest:')
for n in range (10, 51, 10):
    model2 = RandomForestClassifier(random_state = rnd_seed, n_estimators = n)
    model2.fit(train_features, train_target)
    valid_predictions2 = model2.predict(valid_features)
    print('Accuracy score for n = {}:'.format(n), accuracy_score(valid_target, valid_predictions2))
    
print('\n---------------------------------------------\n')

#logistic_regression
print('Logistic Regression:')
model3 = LogisticRegression(random_state = rnd_seed)
model3.fit(train_features, train_target)
valid_predictions3 = model3.predict(valid_features)
print('Accuracy score:', accuracy_score(valid_target, valid_predictions3))

Desigion Tree:
Accuracy score for n = 1: 0.7682737169517885
Accuracy score for n = 2: 0.7916018662519441
Accuracy score for n = 3: 0.8149300155520995
Accuracy score for n = 4: 0.8055987558320373
Accuracy score for n = 5: 0.8133748055987559
Accuracy score for n = 6: 0.807153965785381

---------------------------------------------

Random Forest:
Accuracy score for n = 10: 0.8118195956454122
Accuracy score for n = 20: 0.8133748055987559
Accuracy score for n = 30: 0.8118195956454122
Accuracy score for n = 40: 0.8133748055987559
Accuracy score for n = 50: 0.8118195956454122

---------------------------------------------

Logistic Regression:
Accuracy score: 0.7651632970451011


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


#### Вывод
Из трех принципов конструирования моделей выберем древо решений, поскольку данный способ дает большую точность на валидационной выборке, и модель работает быстрее, нежели случайный лес. Поскольку точность модели, основанной на логистической регрессии слишком близка к требуемой точности, и на тестовой выборке результат скорее всего окажется ниже текущего, мы отвергаем эту модель. (Максимальную глубину древа решений зададим равной 3.)

<div class="alert alert-block alert-success">
<b>Успех:</b> В целом этот шаг сделан хорошо! Исходя из полученных результатов модель и ее гиперпараметр выбраны верно. Молодец!
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> Желательно было бы выделить код для каждой модели в отдельную ячейку.
</div>

## 4. Проверьте модель на тестовой выборке

Обучим выбранную модель на большем объеме данных - на тренировочной и валидационной выборках вместе и узнаем параметр точности. 

In [113]:
final_model = DecisionTreeClassifier(random_state = rnd_seed, max_depth = 3)
final_model.fit(train_total_features, train_total_target)
test_predictions = final_model.predict(test_features)
print('Accuracy score for final_model = {:.2%}'.format(accuracy_score(test_target, test_predictions)))

Accuracy score for final_model = 77.29%


#### Вывод
Выбранная модель смогла достичь точности более 0,75.

<div class="alert alert-block alert-success">
<b>Успех:</b> Абсолютно верно. Поздравляю, тебе удалось достичь хорошего качества!
</div>

## 5. (бонус) Проверьте модели на адекватность

Проверим адекватность модели. Узнаем, какой процент от всех значений целевого признака мы угадаем, заполнив его константным значением большего класса. (В выборке больше пользователей с тарифом "Смарт" - значение целевого признака - 0)

In [109]:
print('{:.2%}'.format(len(test_df.query('is_ultra == 0'))/len(test_df)))

66.87%


#### Вывод
Предсказания модели более точно определяют целевой признак, нежели заполнение его значений константным числом. модель адекватна.

<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично! Все сделано абсолютно верно. В следующих проектах и в релаьных задачах советую всегда делать такую проверку. Замечу, что ее стоит делать до начала обучение моделей.
</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
